# Connectome manipulations & visualizations
This is the main script for connectome manipulations, which
 - applies a single or a sequence of manipulations to the connectome, as specified in the config

> **IMPORTANT:** Use supplementary notebooks
> - `model_building.ipynb` for connectome model building
> - `structural_comparator.ipynb` for structural comparison of the raw and manipulated connectomes

In [1]:
# Initialization

""" Global imports """
import sys
import os.path
import numpy as np

""" Paths """
sys.path.insert(0, '../pipeline/')
sys.path.insert(0, '/gpfs/bbp.cscs.ch/home/pokorny/JupyterLab/git/')

""" Local imports """
from connectome_manipulation import connectome_manipulation
from connectome_comparison import structural_comparator
from model_building import model_building
from topological_sampling.pipeline.gen_topo_db import gen_topo_db
from topological_comparator.pipeline import prepare_pipeline
from topological_comparator.pipeline.additional_stats import add_additional_stats
from topological_comparator.pipeline.contract_db import contract_db
from topological_comparator.pipeline.extract_connectivity import extract_connectivity_sonata
from topological_comparator.bin import compare_topo_db

In [2]:
# Connectome manipulation - Configuration
manip_config = {}

""" Circuit (base path) """
# manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build'
manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1'
# manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/sscx-white-matter'
circuit_name = os.path.split(manip_config['circuit_path'])[1]

""" General settings """
manip_config['seed'] = 3210
manip_config['N_split_nodes'] = 1

""" Manipulation (name + sequence of manipulation functions with arguments) """
# manip_config['manip'] = {'name': 'NoManip', 'fcts': [{'source': 'no_manipulation', 'kwargs': {}}]} # Just for testing

# manip_config['manip'] = {'name': 'RemoveAllSynToL5', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'Layer5', 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'RemoveAllConnToL5', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'Layer5', 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'Remove50pctAllSyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0}}]}
# manip_config['manip'] = {'name': 'Remove50pctAllConn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0}}]}

# manip_config['manip'] = {'name': 'RemoveAllConnWith1To10Syn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'min_syn_per_conn': 1, 'max_syn_per_conn': 10}}]}
# manip_config['manip'] = {'name': 'RemoveAllConnWith5To10Syn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'min_syn_per_conn': 5, 'max_syn_per_conn': 10}}]}
# manip_config['manip'] = {'name': 'RemoveAllConnWithOver20Syn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'min_syn_per_conn': 21}}]}
# manip_config['manip'] = {'name': 'RemoveAllConnExcept5To10Syn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'max_syn_per_conn': 4}},
#                                                                          {'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'min_syn_per_conn': 11}}]}

# manip_config['manip'] = {'name': 'Remove50pctAllSyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0}}]}
# manip_config['manip'] = {'name': 'Remove50pctAllSynKeepGsyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0, 'keep_gsyn': True}}]}
# manip_config['manip'] = {'name': 'Remove50pctAllSynKeepConn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0, 'keep_conns': True}}]}
# manip_config['manip'] = {'name': 'Remove50pctAllSynKeepConnKeepGsyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0, 'keep_conns': True, 'keep_gsyn': True}}]}
# manip_config['manip'] = {'name': 'RemoveAllSynKeepConn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'keep_conns': True}}]}
# manip_config['manip'] = {'name': 'RemoveAllSynKeepConnKeepGsyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'keep_conns': True, 'keep_gsyn': True}}]}
# manip_config['manip'] = {'name': 'ReduceToPointNeuronModel', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'keep_conns': True, 'keep_gsyn': True}},
#                                                                       {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                                       {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_pos', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                                       {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_type', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                                       {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_segment_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                                       {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_segment_offset', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

#manip_config['manip'] = {'name': 'SubsampleSyn10pct', 'fcts': [{'source': 'syn_subsampling', 'kwargs': {'keep_pct': 10.0}}]}

# manip_config['manip'] = {'name': 'RewireAxonsL4PC-L5PC', 'fcts': [{'source': 'axon_rewiring', 'kwargs': {'sel_grp1': {'mtype': ['L4_TPC', 'L4_UPC']}, 'sel_grp2': {'mtype': ['L5_TPC:A', 'L5_TPC:B', 'L5_TPC:C', 'L5_UPC']}, 'R': 100.0, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ShuffleAxonsL4TPC', 'fcts': [{'source': 'axon_shuffling', 'kwargs': {'sel_grp': {'mtype': 'L4_TPC'}, 'R': 100.0, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ShuffleAxonsL456EXC', 'fcts': [{'source': 'axon_shuffling', 'kwargs': {'sel_grp': {'layer': [4, 5, 6], 'synapse_class': 'EXC'}, 'R': 100.0, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ScaleNrrpX2', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'n_rrp_vesicles', 'new_value': {'mode': 'scale', 'factor': 2}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsynX2', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 2}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsynX0.5', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.5}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleDelayX2', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'delay', 'new_value': {'mode': 'scale', 'factor': 2}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleDelayX0.5', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'delay', 'new_value': {'mode': 'scale', 'factor': 0.5}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ScaleGsyn0p1L4aff', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'Layer4', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.1}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsyn0p1L4eff', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'Layer4', 'sel_dest': 'All', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.1}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsyn0p1ExcV1', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_filter': {'syn_type_id': list(range(100, 200))}, 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.1}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsyn0p1ExcV2', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': {'synapse_class': 'EXC'}, 'sel_dest': 'All', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.1}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsyn2p0L4Exc-L4Exc', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': {'layer': 4, 'synapse_class': 'EXC'}, 'sel_dest': {'layer': 4, 'synapse_class': 'EXC'}, 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 2.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPreSynPos', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_x', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_y', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_z', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_x', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_y', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_z', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPreSynSect', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_pos', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_type', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPreSynSegm', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_segment_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_segment_offset', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPostSynPos', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_center_x', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_center_y', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_center_z', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_surface_x', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_surface_y', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_surface_z', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPostSynSect', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                               {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_pos', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                               {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_type', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPostSynSegm', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_segment_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                               {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_segment_offset', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPostSynSectId', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ResetPostSynSectPos', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_pos', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ResetPostSynSectType', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_type', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'InvalPreSynAll', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_x', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_y', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_z', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_x', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_y', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_z', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_id', 'new_value': {'mode': 'setval', 'value': -1}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_pos', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_type', 'new_value': {'mode': 'setval', 'value': -1}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_segment_id', 'new_value': {'mode': 'setval', 'value': -1}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_segment_offset', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ShuffleExc', 'fcts': [{'source': 'axon_shuffling', 'kwargs': {'sel_grp': 'Excitatory', 'R': np.inf, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ConnRewireExcOrd1', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
manip_config['manip'] = {'name': 'ConnRewireOrd1KeepIndeg', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
                                                                     {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd1', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                             {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd2', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                             {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}

# Connectome manipulation - Apply manipulation
connectome_manipulation.main(manip_config, do_profiling=True)


PROFILING: 
************************************
* RESOURCE PROFILING [initial]     *
************************************
* Max. memory usage (GB):    0.121 *
************************************

INFO: Loading circuit from /gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1/sonata/circuit_config.json (N_split=1)
INFO: Split 1/1: Loaded 568717 synapses with 35 properties between 5924 neurons
PROFILING: 
************************************
* RESOURCE PROFILING [loaded-1/1]  *
************************************
* Max. memory usage (GB):    0.216 *
* Max. memory diff. (GB):    0.095 *
************************************
* Total time (s):            0.843 *
* Elapsed time (s):          0.843 *
************************************

INFO: APPLYING MANIPULATION "ConnRewireOrd1KeepIndeg"
INFO: >>Step 1 of 2: source=conn_rewiring, kwargs={'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_di

In [ ]:
assert False, 'STOP HERE'

In [17]:
### ACCESS EDGE TABLE (FOR DEVELOPMENT)
from bluepysnap.circuit import Circuit
from bluepysnap.sonata_constants import Node
from bluepysnap.sonata_constants import Edge

circuit_path = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1'
circuit_name = os.path.split(circuit_path)[-1]
sonata_config =  os.path.join(circuit_path, 'sonata', 'circuit_config.json') # [Using default name]
# sonata_config =  os.path.join(manip_config['circuit_path'], 'sonata', 'circuit_config.json') # [Using default name]
c = Circuit(sonata_config)
nodes = c.nodes['All']
node_ids = nodes.ids()
edges = c.edges['default']
edges_table = edges.afferent_edges(node_ids, properties=sorted(edges.property_names))
circuit = c